# Combined model with BOSS

In [58]:
import pandas as pd
import numpy as np

from tsfresh.feature_extraction import extract_features, EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import matplotlib.pyplot as plt

from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight
from tsfresh.feature_selection import select_features
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate

from pyts.transformation import BOSS

from catboost import CatBoostClassifier

from spark.utils import channel_wise_boss, boss_transform_data
from spark.preprocessor import preprocessor, questionair_preprocessor


In [22]:
raw_data_prep_dir = '../raw_data/pads-parkinsons-disease-smartwatch-dataset-1.0.0/preprocessed/'

In [23]:
file_list = pd.read_csv(raw_data_prep_dir  + "file_list.csv")
file_list

,resource_type,id,study_id,condition,disease_comment,age_at_diagnosis,age,height,weight,gender,handedness,appearance_in_kinship,appearance_in_first_grade_kinship,effect_of_alcohol_on_tremor,label
0,patient,1,PADS,Healthy,-,56,56,173,78,male,right,True,True,Unknown,0
1,patient,2,PADS,Other Movement Disorders,Left-Sided resting tremor and hypokinesia with...,69,81,193,104,male,right,False,NaN,No effect,2
2,patient,3,PADS,Healthy,-,45,45,170,78,female,right,False,NaN,Unknown,0
3,patient,4,PADS,Parkinson's,IPS akinetic-rigid type,63,67,161,90,female,right,False,NaN,No effect,1
4,patient,5,PADS,Parkinson's,IPS tremordominant type,65,75,172,86,male,left,False,NaN,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,patient,465,PADS,Parkinson's,IPS mixed type,62,65,175,80,male,right,True,False,No effect,1
465,patient,466,PADS,Healthy,-,84,84,172,74,female,right,True,True,No effect,0
466,patient,467,PADS,Parkinson's,"Essential Tremor, starting IPS tremordominant ...",55,57,190,100,male,right,False,NaN,Improvement,1
467,patient,468,PADS,Parkinson's,IPS mixed type,73,76,198,118,male,right,False,NaN,No effect,1


In [61]:
X_train_q, X_test_q, y_train_q, y_test_q = preprocessor("~/code/marzecovaa/spark/processed_data/merged_dfq.csv")

✅ Transformer saved at: ./model_registry/transformer_20250908-121848.joblib


/Users/anna/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [24]:
time_data = []
y = []
X_to_split = []
for idx, subject_idx in enumerate(file_list['id']):
    file_name = raw_data_prep_dir + f'movement/{subject_idx:03d}_ml.bin'
    time_idx_data = np.fromfile(file_name, dtype=np.float32).reshape((-1, 976))
    time_data.append(time_idx_data)
    y_idx = file_list[file_list['id']==subject_idx].label.values
    X_id = idx
    y.append(y_idx)
    X_to_split.append(X_id)

X_to_split = np.array(X_to_split)
y = np.array(y).flatten()
time_data = np.array(time_data)
print(time_data.shape)
time_data.shape[0]
range(time_data.shape[0])

(469, 132, 976)


range(0, 469)

In [6]:
#train test split
X_train_idx, X_test_idx, y_train, y_test = train_test_split(X_to_split, y, test_size=0.20, random_state=42, stratify = y)
X_train = time_data[X_train_idx]
X_test = time_data[X_test_idx]
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))
X_train.shape

375 94
375 94


(375, 132, 976)

In [36]:
boss = channel_wise_boss(X_train, n_bins = 4, window_step = 2, window_size = 10, word_size = 6) #Save this to transform

BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(window_step=2, word_size=6)
BOSS(windo

In [41]:
X_train_boss = boss_transform_data(boss,X_train)

1


In [45]:
X_test_boss = boss_transform_data(boss,X_test)

1


In [46]:
pd.DataFrame(X_train_boss)

,0,1,2,3,4,5,6,7,8,9,...,378434,378435,378436,378437,378438,378439,378440,378441,378442,378443
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
373,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
pca = PCA(n_components = 100)

In [48]:
X_pca_train = pca.fit_transform(X= X_train_boss)
X_pca_test = pca.transform(X=X_test_boss)
X_pca_train.shape

(375, 100)

In [49]:
cat_model_pca = CatBoostClassifier(iterations = 2000,
                                    learning_rate = 0.05,
                                    depth = 7)

In [50]:
sample_weights = compute_sample_weight(class_weight="balanced", y=y_train)

cat_model_pca.fit(X_pca_train, y_train,
                  eval_set = (X_pca_test, y_test),
                  use_best_model = True,
                  sample_weight=sample_weights)

0:	learn: 1.0883321	test: 1.0961222	best: 1.0961222 (0)	total: 29.2ms	remaining: 58.3s
1:	learn: 1.0730500	test: 1.0882599	best: 1.0882599 (1)	total: 40.6ms	remaining: 40.6s
2:	learn: 1.0636668	test: 1.0875448	best: 1.0875448 (2)	total: 51.5ms	remaining: 34.3s
3:	learn: 1.0528854	test: 1.0848777	best: 1.0848777 (3)	total: 61.9ms	remaining: 30.9s
4:	learn: 1.0425464	test: 1.0824489	best: 1.0824489 (4)	total: 72.9ms	remaining: 29.1s
5:	learn: 1.0342980	test: 1.0806882	best: 1.0806882 (5)	total: 85.3ms	remaining: 28.3s
6:	learn: 1.0241985	test: 1.0773236	best: 1.0773236 (6)	total: 96ms	remaining: 27.3s
7:	learn: 1.0121015	test: 1.0759289	best: 1.0759289 (7)	total: 108ms	remaining: 26.8s
8:	learn: 1.0029669	test: 1.0729773	best: 1.0729773 (8)	total: 120ms	remaining: 26.6s
9:	learn: 0.9920168	test: 1.0712177	best: 1.0712177 (9)	total: 131ms	remaining: 26s
10:	learn: 0.9838874	test: 1.0681881	best: 1.0681881 (10)	total: 143ms	remaining: 25.9s
11:	learn: 0.9751953	test: 1.0649176	best: 1.0649

In [51]:
y_pred_cat = cat_model_pca.predict(X_pca_test)

In [52]:
cat_accuracy = balanced_accuracy_score(y_test, y_pred_cat)
f'cat_accuracy: {cat_accuracy}'

'cat_accuracy: 0.5566534914361002'

In [53]:
sv_model_pca = SVC(kernel = 'rbf', class_weight = 'balanced', C = 1)
sv_model_pca.fit(X_pca_train, y_train)

,C,1
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,'balanced'
,verbose,False


In [54]:
y_pred_pca_sv = sv_model_pca.predict(X_pca_test)
balanced_accuracy_score(y_test, y_pred_pca_sv)

0.7275197628458497

In [ ]:
pca_q = PCA(n_components = 30)


(375, 413)